In [3]:
import boto3
import json
import numpy as np
import textwrap
from sklearn.metrics.pairwise import cosine_similarity

# Initialize Bedrock client
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")

# Define the prompt
prompt_text = "Write a short paragraph about the future of artificial intelligence."

# Function to get text embeddings using Bedrock
def get_text_embedding(text):
    payload = {"inputText": text}

    response = bedrock_runtime.invoke_model(
        modelId="amazon.titan-embed-text-v2:0",  # Embedding model
        contentType="application/json",
        accept="application/json",
        body=json.dumps(payload),
    )

    response_body = json.loads(response["body"].read())
    embedding = response_body.get("embedding", [])
    return np.array(embedding, dtype=float)

# Text generation (Claude) — now accepts optional temperature param
def generate_response(top_p_value: float, temperature: float = 0.7) -> str:
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt_text}],
            }
        ],
        "max_tokens": 200,
        "temperature": temperature,
        "top_p": top_p_value,
        "stop_sequences": ["\n\nHuman:"],
    }

    resp = bedrock_runtime.invoke_model(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body),
    )

    resp_body = json.loads(resp["body"].read())
    # Anthropic Messages API returns a list of content blocks
    text_chunks = [
        block.get("text", "")
        for block in resp_body.get("content", [])
        if block.get("type") == "text"
    ]
    return textwrap.fill("".join(text_chunks).strip(), width=80)

# Generate responses with different top_p / temperature values
high_top_p_response = generate_response(0.9, 0.7)   # High top_p and default temperature
low_top_p_response = generate_response(0.1, 0.7)    # Low top_p and default temperature
medium_top_p_response = generate_response(0.5, 0.9) # Medium top_p with higher temperature

# Get embeddings for all three responses
high_top_p_embedding = get_text_embedding(high_top_p_response)
low_top_p_embedding = get_text_embedding(low_top_p_response)
medium_top_p_embedding = get_text_embedding(medium_top_p_response)

# Compute cosine similarity between the three embeddings (sklearn)
similarity_high_low = cosine_similarity([high_top_p_embedding], [low_top_p_embedding])
similarity_high_medium = cosine_similarity([high_top_p_embedding], [medium_top_p_embedding])
similarity_low_medium = cosine_similarity([low_top_p_embedding], [medium_top_p_embedding])

# Print the responses and similarity scores
print("\n### Response with High top_p (0.9) - More Creative & Diverse:\n")
print(high_top_p_response)

print("\n### Response with Low top_p (0.1) - More Predictable & Focused:\n")
print(low_top_p_response)

print("\n### Response with Medium top_p (0.5) and Temperature (0.9) - Balanced:\n")
print(medium_top_p_response)

print("\n### Cosine Similarity between high top_p (0.9) and low top_p (0.1) outputs:")
print(f"Cosine Similarity: {similarity_high_low[0][0]:.4f}")

print("\n### Cosine Similarity between high top_p (0.9) and medium top_p (0.5) outputs:")
print(f"Cosine Similarity: {similarity_high_medium[0][0]:.4f}")

print("\n### Cosine Similarity between low top_p (0.1) and medium top_p (0.5) outputs:")
print(f"Cosine Similarity: {similarity_low_medium[0][0]:.4f}")



### Response with High top_p (0.9) - More Creative & Diverse:

The future of artificial intelligence (AI) holds immense potential and promises
to revolutionize various aspects of our lives. As AI technologies continue to
advance, we can expect to see more sophisticated and intelligent systems that
can tackle complex tasks with greater efficiency and accuracy. From healthcare
to transportation, education to entertainment, AI will play a pivotal role in
enhancing our experiences and solving challenging problems. However, it is
crucial to address ethical concerns and ensure that AI development is guided by
principles of transparency, accountability, and respect for human rights. With
responsible development and implementation, AI has the power to unlock new
frontiers of innovation and create a better, more sustainable future for
humanity.

### Response with Low top_p (0.1) - More Predictable & Focused:

The future of artificial intelligence (AI) holds immense potential and promises
to re